# Pset #1b part 2 of 2: inverse kinematics (IK) of planar arms

Inverse kinematics (IK) is the problem of finding the joint angles that give desired end effector (EE) postures. We will investigate different ways to solve IK of a 2-link arm and a 3-link arm. 

Here's a schematic diagram of the two-link arm we'll be using in this notebook.
<img src="two_link_arm.png" alt="drawing" width="500"/>

And here's the three-link arm. 
<img src="three_link_arm.png" alt="drawing" width="500"/>

In [ ]:
# Import stuff to be used in this notebook. Run this first!
%reload_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import numpy as np

from inverse_kin_planar_arm import (ForwardKinematics,
                                    ForwardKinematicsThreeLink,
                                    InverseKinByTrigonometry,
                                    InverseKinByGradientDescent,
                                    InverseKinByGradientDescentThreeLink,
                                    IkThreeLinkArmMathematicalProgram,
                                    l1, l2,)


## IK by trigonometry (2-link arm)

We want to find $\theta = [\theta_1, \theta_2]$ that places the end effector of the 2-link robot at $x_{ee_{d}} \in \mathbb{R}^2$. For something as simple as a 2-link arm, solving IK constitutes an interesting exercise in trigonometry. Complete the function `InverseKinByTrigonometry` in `inverse_kin_planar_arm.py`, and check that it indeed places the robot's EE at the desired position.  Note the definition of the link lengths $l_i$ at the top of the file.

In [ ]:
x_ee_desired = np.array([0.5, 0.2])

theta = InverseKinByTrigonometry(x_ee_desired)
print "joint angles:", theta
print "end effector position: ", ForwardKinematics(theta)

## IK by gradient descent (2-link arm)

Getting analytic IK solutions becomes horrifying as the number of links in the arm increases. Moreover, it is hard to incorporate into analytic solutions constraints such as joint limits.  And, importantly, when there are more joints than constraints from the end effector, as in 7DOF arms, there are infinitely many solutions.

A more practical way to solve IK for 7DOF arms in 3D is to formulate the IK problem as an optimization. A common choice for the objective function to minimize is some sort of "error" between the current and desired EE posture.

Your task is to 
- Complete function `Objective` in `inverse_kin_planar_arm.py`. Use $\|x_{ee} - x_{ee_d}\|^2$ as the error to minimize. $x_{ee}$ is the optimization's decision variable, and $x_{ee_d}$ is the desired EE position. 
- Write down the gradient of the objective function (complete `GradientObjective` in `inverse_kin_planar_arm.py`).
- Pass `Objective`, `GradientObjective` and your initial guess to the `GradientDescent` function you completed in part 1 of this pset, which should return a set of joint angles that minimizes the `Objective`. 

Your optimization should not need any constraints.

In [ ]:
# Set desired EE position
x_ee_desired = np.array([0.2, 0.6])

# initial guess
theta0 = np.array([2, -2])

# gradient descent
theta, error_value, Objective = InverseKinByGradientDescent(x_ee_desired, theta0)
print "theta: ", theta
print "error value: ", error_value
print "EE position: ", ForwardKinematics(theta)

# Generate data for plotting
n = 50
x = np.linspace(-np.pi + theta[0], np.pi + theta[0], n)
y = np.linspace(-np.pi + theta[1], np.pi + theta[1], n)
xv, yv = np.meshgrid(x,y)
zv = np.zeros((n, n))

for i, xi in enumerate(x):
    for j, yi in enumerate(y):
        zv[i,j] = Objective([xi, yi])

plt.contour(xv, yv, zv, 50)
plt.plot(theta[0], theta[1], '*')

### Exploring local minima
As the objective function is non-convex (it has sin's and cos'), it could have multiple local minima, as shown in the contour plot generated above. The multiple local minima correspond to multiple solutions to the IK problem: if an IK problem of a planar arm has one solution, another solution can be readily obtained by mirroring the arm about the line connecting the origin and the EE. 

The local minimum at which gradient descent ends up depends heavily on where the descent starts (your initial guess). You are asked to find two initial guesses that converge to two distinct local minima (joint angles). Edit the `theta_0` and `theta_1` parameters in `InverseKinLocalMinimum()` in `inverse_kin_planar_arm.py`. The code and tests have been replicated below for your convenience, but **you will need to edit the variables in the .py file for the graded tests to pass**.

In [ ]:
# Set desired EE position
x_ee_desired = np.array([0.3, 0.6])

theta_0 = np.array([0,0])
theta_1 = np.array([0,0])

initial_guesses = [theta_0, theta_1]
solutions = np.zeros((2,2))

for i in range(2):
    solutions[i], _, _ = InverseKinByGradientDescent(x_ee_desired, initial_guesses[i])
    
# verify solutions
for theta in solutions:
    print "theta: ", theta
    print "EE position: ", ForwardKinematics(theta)

## IK by gradient descent (3-link arm)

For a 3-link planar arm, if we only specify the end-point position $x_{ee}$, the the arm is "redundant" and there are in general a continuous family of solutions.  In that situation, we could change the objective function (and its gradient) so that you can specify that you want a solution where the joint angles are "close" to a specified set of joint angles, in practice this might be the ones where the robot is currently placed.  You should try that.

Alternatively, we could also specify the orientation of the last link, in which case there would be a finite number of solutions.  

Complete the function `ForwardKinematicsThreeLink` that takes in 3 joint angles and returns a vector of the end effector position and also the angle of the last link.  Now define the `ObjectiveThreeLink` function and `GradientObjectiveThreeLink` so as to be able to use gradient descent to solve IK for the 3-link arm.

In [ ]:
# Set desired EE position, x3_ee_desired[3] is the desired end effector orientation.
x3_ee_desired = np.array([0.3, 0.6, 0])

# initial guess
theta0 = np.array([2.,-2., 1.])

# gradient descent
theta, error_value, Objective = InverseKinByGradientDescentThreeLink(x3_ee_desired, theta0)
print "theta: ", theta
print "error value: ", error_value
print "EE position: ", ForwardKinematicsThreeLink(theta)

## IK by drake's optimization interface: MathematicalProgram (2 and 3-link arms)

We have been posing IK as an unconstrained optimization problem of minimizing an error function, which may be mixing apples (position error) and oranges (orientation error).  We also saw that in the "redundant" case, we might want to consider other criteria, such as being close to a desired solution.  In general, we can resolve these issues by using a constrained formulation, where we minimize an objective subject to equality or inequality constraints.  Using 2-link IK as an example, we will demonstrate how to formulate and solve optimization problems using Drake's `MathematicalProgram` interface. In addition to the text and code below, you can learn more about optimization and `MathematicalProgram` in the `mathematical_program_examples.ipynb` notebook.

`MathematicalProgram` uses [SNOPT](https://en.wikipedia.org/wiki/SNOPT), a powerful commercial solver, to solve nonlinear, nonconvex optimizations. SNOPT also uses descent to find local minima, but the descent direction is more sophisticated than the gradient of the objective function. 

As SNOPT can handle constraints, we will formulate IK as a slightly different optimization problem this time around. Instead of minimizing the "error", we will set EE position as constraints, and the 2-norm of the joint anlges the objective function: 

$$ 
\begin{align}
\mathrm{minimize}&& ~ \theta_1^2 + \theta_2^2 \\
\mathrm{subject \, to} 
&& x_{ee_d}[0] = l_1\cos(\theta_1) + l_2\cos(\theta_1 + \theta_2) \\
&& x_{ee_d}[1] = l_1\sin(\theta_1) + l_2\sin(\theta_1 + \theta_2)
\end{align}
$$

Constraints can also be used to meet requirements that simple gradient descent cannot satisfy. For example, we can ask the first link to stay within the first two quadrants by adding the following constraint:
$$
0 \leq \theta_1 \leq \pi
$$

The following block illustrates how to construct, solve and retrieve the solution of the above IK problem using `MathematicalProgram`.

In [ ]:
from pydrake.all import MathematicalProgram
import pydrake.symbolic as sym

theta_initial_guess = np.array([np.pi/6, -np.pi/3])
x_ee_desired = np.array([0.3, 0.5])

# create a mathematical program object.
prog = MathematicalProgram()

# create decision variables
theta = prog.NewContinuousVariables(2, 'theta')

# Add constraints
# Every element of theta is a drake symbolic object, which numpy's sin and cos cannot handle.
# Therefore, sym.cos or sym.sin are used to express the constraints, so that the expression 
# makes sense to MathematicalProgram. 
prog.AddConstraint(l1*sym.cos(theta[0]) + l2*sym.cos(theta[0] + theta[1]) == x_ee_desired[0])
prog.AddConstraint(l1*sym.sin(theta[0]) + l2*sym.sin(theta[0] + theta[1]) == x_ee_desired[1])
prog.AddLinearConstraint(theta[0] >= 0)
prog.AddLinearConstraint(theta[0] <= np.pi)

# Add objective function
prog.AddQuadraticCost((theta**2).sum())

# set initial guess
prog.SetInitialGuess(theta, theta_initial_guess)
result = prog.Solve()
theta_value = prog.GetSolution(theta)

# verify results
print "Solution result: ", result
print "IK solution: ", theta_value
print "EE position: ", ForwardKinematics(theta_value)

### Solving IK for a 3-link arm with MathematicalProgram

Your task is to formulate and solve an IK problem for a 3-link arm using `MathematicalProgram` (complete `IkThreeLinkArmMathematicalProgram` in `inverse_kin_planar_arm.py`). The solution should satisfy the following constraints:
- $x_{ee} = [1.0, 1.0]$
- $\theta_{ee} = \pi/2$
- the first link should stay within the first or the fourth quadrant. 

In [ ]:
x_ee_desired = np.array([1.0, 1.0])
theta_ee_desired = np.pi/2
theta_initial_guess = np.zeros(3)

theta_value, result = \
    IkThreeLinkArmMathematicalProgram(x_ee_desired, theta_ee_desired, theta_initial_guess)

# verify results
print "Solution result: ", result
print "IK solution: ", theta_value
print "EE position: ", ForwardKinematicsThreeLink(theta_value)

## IK by Drake's IK interface

Drake has a wrapper class `InverseKinematics` around `MathematicalProgram` that facilitates adding constraints commonly used in IK problems. Here's the [documentation](http://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_inverse_kinematics.html) on the **quickly growing** list of constraints that `InverseKinematics` supports. 

The cell below is an example of using `InverseKinematics` on the two-link arm.

Note that because of the SDF (similar to URDF but a lot more confusing) files we are using, **the arms now live in the YZ plane instead of the XY plane.** 

In [ ]:
from inverse_kin_planar_arm import two_link_arm_path, ForwardKinematicsMbp
from pydrake.multibody.multibody_tree.multibody_plant import MultibodyPlant
from pydrake.geometry import SceneGraph
from pydrake.multibody.multibody_tree.parsing import AddModelFromSdfFile
from pydrake.multibody import inverse_kinematics as ik

# construct multibodyplant
timestep = 0.0002
mbp = MultibodyPlant(timestep)
scene_graph = SceneGraph()

# Add models
arm_model = AddModelFromSdfFile(
    file_name=two_link_arm_path, model_name='robot',
    scene_graph=scene_graph, plant=mbp)

# finalize multibodyplant
mbp.Finalize(scene_graph)
assert mbp.geometry_source_is_registered()

# create an instance of InverseKinematics from a multibodyplant.
ik_two_link_arm = ik.InverseKinematics(mbp)

world_frame = mbp.world_frame()
link_ee_frame = mbp.GetBodyByName("link_ee").body_frame()

p_WQ = np.array([0, 0.5, 0.5])
p_EQ = np.zeros(3)
p_EQ_lower_bound = p_EQ - 0.01
p_EQ_upper_bound = p_EQ + 0.01

ik_two_link_arm.AddPositionConstraint(
    frameB=world_frame, p_BQ=p_WQ,
    frameA=link_ee_frame,
    p_AQ_lower=p_EQ_lower_bound, p_AQ_upper=p_EQ_upper_bound)

# NB: an instance of InverseKinematics has an underlying MathematicalProgram. 
prog = ik_two_link_arm.get_mutable_prog()
result = prog.Solve()
theta_solution = prog.GetSolution(ik_two_link_arm.q())

print "IK solution result:", result
print "jonit angles:", theta_solution
print "desired ee position:", p_WQ
print "actual ee position:", ForwardKinematicsMbp(theta_solution, link_ee_frame, mbp)


### Solving IK for a 3-link arm with InverseKinematics

Your task is to formulate and solve an IK problem for a 3-link arm using `InverseKinematics` (complete `IkThreeLinkArmMathematicalProgram` in `inverse_kin_planar_arm.py`). The solution should satisfy the following constraints:
- Point Q, which is rigidly attached to the end effector frame E at p_EQ = \[0.0, 0.1, 0.1\] (p_EQ reads position of point Q relative to frame E), is at $[0, 1.2, 1.5]$ in the world frame, i.e. p_WQ = \[0.0, 1.2, 1.5\] 
- The joint angle of the link 1 is $\pi/6$, i.e. $\theta[0] = \pi/6$.

In [ ]:
from inverse_kin_planar_arm import IkThreeLinkArmByDrakeInverseKinematics, ForwardKinematicsPointMbp
theta_solution, result, mbp_3_link = IkThreeLinkArmByDrakeInverseKinematics()
p_EQ = np.array([0.0, 0.1, 0.1])
frame = mbp_3_link.GetBodyByName("link_ee").body_frame()
print "IK solution result:", result
print "jonit angles:", theta_solution
print "desired ee position:", [0, 1.2, 1.5]
print "actual ee position:", ForwardKinematicsPointMbp(p_EQ,theta_solution, frame, mbp_3_link)

# Run Tests
Running the cell below will print out a score for every test and the total score.

In [ ]:
import os
# Run the tests
os.popen("python test_pset_1b.py test_results.json")

# Print the results json for review
import test_pset_1b
print test_pset_1b.pretty_format_json_results("test_results.json")